In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It", groq_api_key = groq_api_key)

In [7]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content= "Hi, I am Pranav and a data scientiest")])

AIMessage(content="Hi Pranav! It's nice to meet you. \n\nAs a large language model, I'm always interested in learning about what data scientists are working on. What kind of projects are you involved in these days?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 20, 'total_tokens': 69, 'completion_time': 0.089090909, 'prompt_time': 0.001923336, 'queue_time': 0.253568744, 'total_time': 0.091014245}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c0e80a5a-5f07-44f3-bddc-51e299100973-0', usage_metadata={'input_tokens': 20, 'output_tokens': 49, 'total_tokens': 69})

In [8]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage("Hi, I am Pranav and a data scientiest"),
    AIMessage(content="Hi Pranav! \n\nIt's nice to meet you.  \n\nWhat kind of data science work are you involved in? I'm always interested to learn about what people are doing with data. 😄 \n\n"),
    HumanMessage(content="Hey whats my name and profession?")
    
])

AIMessage(content="You are Pranav, and you are a data scientist!  \n\nYou told me so at the beginning of our conversation. 😊  \n\nIs there anything else you'd like to talk about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 82, 'total_tokens': 125, 'completion_time': 0.078181818, 'prompt_time': 0.007743491, 'queue_time': 0.254553338, 'total_time': 0.085925309}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--6b6c32d6-882d-4937-acd5-d2f95cda3c09-0', usage_metadata={'input_tokens': 82, 'output_tokens': 43, 'total_tokens': 125})

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_hist = RunnableWithMessageHistory(model, get_session)

In [22]:
config = {'configurable' : {"session_id": "Chat1"}}

In [23]:
response = with_message_hist.invoke(

    [ HumanMessage('Hi, I am Pranav and a data scientiest')],
      config= config

)

In [25]:
response.content

"Hi Pranav! It's nice to meet you. \n\nWhat can I do for you today? Are you working on any interesting projects? Perhaps you have a data science question you'd like to discuss, or maybe you just want to chat about the field?  \n\nI'm here to help in any way I can! 😄\n"

In [26]:
response = with_message_hist.invoke(

    [ HumanMessage('Whats my name')],
      config= config

)

response.content

'Your name is Pranav, as you told me at the beginning of our conversation.  😊 \n\nIs there anything else I can help you with?\n'

In [27]:
config1 = {'configurable' : {"session_id": "Chat2"}}

response = with_message_hist.invoke(

    [ HumanMessage('Whats my name')],
      config= config1

)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know!\n"

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(

    [
        ('system', 'You are a AI assistant answer all questions to the best of your abilities'),
        MessagesPlaceholder(variable_name='messages')

    ]

)

parser = StrOutputParser()

chain = prompt | model | parser

In [10]:
response = chain.invoke({'messages': [HumanMessage(content= "Hi I am Pranav")]})
response

"Hi Pranav! 👋\n\nIt's nice to meet you.  \n\nI'm ready to answer your questions to the best of my ability.  What can I help you with today? 😊  \n\n"

In [34]:
with_message_hist = RunnableWithMessageHistory(chain, get_session)

In [37]:
config2 = {'configurable' : {'session_id':"Chat3"}}


response = with_message_hist.invoke(
    [HumanMessage(content= "Whats my name?")],
    config = config2
)

response


AIMessage(content="Since I don't have access to past conversations, I still don't know your name.  \n\nWould you like to tell me?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 79, 'total_tokens': 113, 'completion_time': 0.061818182, 'prompt_time': 0.004260329, 'queue_time': 0.24883381100000002, 'total_time': 0.066078511}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--592d28fe-4f07-4815-ab26-c45449f58dc7-0', usage_metadata={'input_tokens': 79, 'output_tokens': 34, 'total_tokens': 113})

In [43]:
prompt = ChatPromptTemplate.from_messages(

    [
        ('system', 'You are a AI assistant answer all questions to the best of your abilities in {language}'),
        MessagesPlaceholder(variable_name='messages')

    ]

)

chain = prompt | model

In [44]:
response = chain.invoke({'messages': [HumanMessage(content= "Hi I am pranav")], "language":"Hindi"})
response

AIMessage(content='नमस्ते प्रणव! 👋 \n\nमैं आपकी सहायता करने के लिए तैयार हूँ। आप मुझसे जो भी प्रश्न पूछेंगे, मैं अपनी पूरी क्षमता से उत्तर देने की कोशिश करूंगा।  😊 \n\nआपका कोई प्रश्न है? 🤔\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 30, 'total_tokens': 97, 'completion_time': 0.121818182, 'prompt_time': 0.005313257, 'queue_time': 0.257914323, 'total_time': 0.127131439}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c33fbb3c-2bb5-4e61-855d-550f34393ba4-0', usage_metadata={'input_tokens': 30, 'output_tokens': 67, 'total_tokens': 97})

In [49]:
with_message_hist = RunnableWithMessageHistory(
    chain,
    get_session,
    input_messages_key= 'messages'
)
config3 = {'configurable' : {'session_id':"Chat4"}}

response = with_message_hist.invoke(
    {'messages':[HumanMessage(content= "Hi I am pranav")], "language":"Hindi"},
    config=config
)
response.content


'प्रणव जी, नमस्ते!  \n\nआप कैसे हैं? क्या मैं आपकी कोई मदद कर सकता हूँ?  😊  \n'

In [50]:
response = with_message_hist.invoke(
    {'messages':[HumanMessage(content= "Whats my name?")], "language":"Hindi"},
    config=config
)
response.content


'आपका नाम प्रणव है। 😊  \n'